## 데이터를 가공하고 저장합니다.

현재 위치에서 "../../data/" 에 2019-Oct.csv, 2019-Nov.csv 파일이 있어야한다

In [1]:
import pandas as pd
import datetime
from my_function import prepro
from my_function import to_hour_category
DATA_PATH = "../../data/"
data_list = ["2019-Oct", "2019-Nov"]

In [ ]:
# csv 파일을 추후 핸들링을 위해 parquet 으로 저장
for data in data_list:
    df = pd.read_csv(DATA_PATH + data + ".csv")
    df.to_parquet(DATA_PATH + data + ".parquet.gzip")
    del df

In [ ]:
# 각 데이터마다 기본적인 전처리후 전처리된 데이터프레임을 after_prepro_를 붙여서 parquet 으로 저장
# product_id, category_id, user_session 제외
for data in data_list:
    df = pd.read_parquet(DATA_PATH + data + ".parquet.gzip", columns = ["event_time", "event_type", "category_code", "brand", "price", "user_id"])
    df = prepro(df)
    df.to_parquet(DATA_PATH + "after_prepro_" + data + ".parquet.gzip")
    del df

In [ ]:
# 각각 결제한 로그, 장바구니 담은 로그, 열람 로그 로 따로 저장
for data in data_list:
    for event in ["purchase", "cart", "view"]:
        df = pd.read_parquet(DATA_PATH + "after_prepro_" + data + ".parquet.gzip")
        df = df[df[f"event_type_{event}"] == 1]
        df.to_parquet(DATA_PATH + event + "_" + data + ".parquet.gzip")
        del df

In [ ]:
# 10월과 11월 데이터의 합을 각각 불러오고 합쳐서 Oct_Nov.parquet.gzip으로 저장
df_10 = pd.read_csv(DATA_PATH + "2019-Oct.csv")
df_11 = pd.read_csv(DATA_PATH + "2019-Nov.csv")
df = pd.concat([df_10, df_11])
df.to_parquet(DATA_PATH + "Oct_Nov.parquet.gzip")
del df_10
del df_11
del df

In [2]:
# 각각 결제한 로그, 장바구니 담은 로그, 열람 로그 로 따로 저장
df = pd.read_parquet(DATA_PATH + "Oct_Nov.parquet.gzip")
for event in ["purchase", "cart", "view"]:
    df[df["event_type"] == event].to_parquet(DATA_PATH + f"{event}_" +"Oct_Nov.parquet.gzip")

In [31]:
# 결제, 장바구니, 열람로그별 상위 10개의 카테고리를 중복없이 top10이라는 리스트를 확보
top10 = []
for event in ["purchase", "cart", "view"]:
    df = pd.read_parquet(DATA_PATH + f"{event}_" +"Oct_Nov.parquet.gzip", columns = ["category_code"])
    top10 += df["category_code"].value_counts()[:10].index.to_list()
top10 = list(set(top10))

# 1차 카테고리의 전체 리스트 (big_category) 확보
df = pd.read_parquet(DATA_PATH + "Oct_Nov.parquet.gzip", columns = ["category_code"])
df = df.fillna("missing")
df["category_code_0"] = df["category_code"].apply(lambda x : x.split(".")[0])
big_category = df["category_code_0"].value_counts().index.to_list()
big_category.remove("missing")

In [32]:
# 시간별 데이터를 생성하는 함수
def convert_to_time_data(data):
    import pandas as pd 

    # 불러오는 data frame은 my_function.prepro() 함수를 통해 1차로 전처리를 실시한 상태
    df = pd.read_parquet(f"../../data/after_prepro_{data}.parquet.gzip", columns=["month", "day", "hour", "event_type_view", "event_type_purchase",	"event_type_cart"])
    df["event_type_total"] = df["event_type_view"] + df["event_type_purchase"] + df["event_type_cart"]

    # 시간별 유저의 행동 (보기, 장바구니, 결제, 전체) 변수 (event_type_view, event_type_cart, event_type_purchase, event_type_total) 생성
    temp_df = df.groupby(["month", "day", "hour"]).sum()

    # 시간별 전체 매출액 변수(total_sales) 생성
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "price"])
    temp_df = pd.concat([temp_df, df.groupby(["month", "day", "hour"]).sum()], axis=1)
    temp_df.rename(columns={"price" : "total_sales"}, inplace = True)

    # 시간별 접속 유저수 변수 (no_of_total_user) 생성
    df = pd.read_parquet(f"../../data/after_prepro_{data}.parquet.gzip", columns=["month", "day", "hour", "user_id"])
    temp_df = pd.concat([temp_df, df.groupby(["month", "day", "hour"]).nunique()], axis=1)
    temp_df.rename(columns={"user_id" : "no_of_total_user"}, inplace = True)

    # 시간별 장바구니 담기를 실시한 유저수 변수 (no_of_cart_user) 생성
    df = pd.read_parquet(f"../../data/cart_{data}.parquet.gzip", columns=["month", "day", "hour", "user_id"])
    temp_df = pd.concat([temp_df, df.groupby(["month", "day", "hour"]).nunique()], axis=1)
    temp_df.rename(columns={"user_id" : "no_of_cart_user"}, inplace = True)

    # 시간별 결제를 실시한 유저수 변수 (no_of_purchase_user) 생성
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "user_id"])
    temp_df = pd.concat([temp_df, df.groupby(["month", "day", "hour"]).nunique()], axis=1)
    temp_df.rename(columns={"user_id" : "no_of_purchase_user"}, inplace = True)

    # 시간별 카테고리별 결제횟수변수 (purchase_of_categroy) 생성
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code"])
    for category in top10:
        category_df = df[df["category_code"] == category]
        temp_df = pd.concat([temp_df, category_df.groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code" : f"purchase_of_{category}"}, inplace = True)

    # 시간별 카테고리별 매출액 (sales_of_categroy) 생성      
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "price","category_code"])
    for category in top10:
        category_df = df[df["category_code"] == category]
        temp_df = pd.concat([temp_df, category_df[["month", "day", "hour", "price"]].groupby(["month", "day","hour"]).sum()], axis=1)
        temp_df.rename(columns={"price" : f"sales_of_{category}"}, inplace = True)    

    # 시간별 1차 카테고리별 결제 횟수 (sales_of_categroy) 생성
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code_0"])
    for category in big_category:
        category_df = df[df["category_code_0"] == category]
        category_df = category_df[["month", "day", "hour", "category_code_0"]]
        temp_df = pd.concat([temp_df, category_df.groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code_0" : f"purchase_of_st_{category}"}, inplace = True)

    # 시간별 1차 카테고리별 매출액 (sales_of_categroy) 생성
    # purchase_data.parquet.gzip 은 전처리 후 결제가 이루어진 로그만 가지고 있는 data
    df = pd.read_parquet(f"../../data/purchase_{data}.parquet.gzip", columns=["month", "day", "hour", "price","category_code_0"])
    for category in big_category:
        category_df = df[df["category_code_0"] == category]
        temp_df = pd.concat([temp_df, category_df[["month", "day", "hour", "price"]].groupby(["month", "day","hour"]).sum()], axis=1)
        temp_df.rename(columns={"price" : f"sales_of_st_{category}"}, inplace = True)        

    # 시간별 매달 장바구니 담은 기준 상위 5개의 카테고리별 장바구니 담기 횟수변수 (cart_of_categroy) 생성
    df = pd.read_parquet(f"../../data/cart_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code"])
    for category in top10:
        category_df = df[df["category_code"] == category]
        category_df = category_df[["month", "day", "hour", "category_code"]]
        temp_df = pd.concat([temp_df, category_df.groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code" : f"cart_of_{category}"}, inplace = True)

    # 시간별 1차 카테고리별 장바구니 담기 횟수변수 (cart_of_st_categroy) 생성
    df = pd.read_parquet(f"../../data/cart_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code_0"])
    for category in big_category:
        category_df = df[df["category_code_0"] == category]
        temp_df = pd.concat([temp_df, category_df[["month", "day", "hour", "category_code_0"]].groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code_0" : f"cart_of_st_{category}"}, inplace = True)

    # 시간별 매달 view 기준 상위 5개의 카테고리별 view 횟수변수 (view_of_categroy) 생성
    df = pd.read_parquet(f"../../data/view_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code"])
    for category in top10:
        category_df = df[df["category_code"] == category]
        category_df = category_df[["month", "day", "hour", "category_code"]]
        temp_df = pd.concat([temp_df, category_df.groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code" : f"view_of_{category}"}, inplace = True)

    # 시간별 1차 카테고리별 view 횟수변수 (view_of_st_categroy) 생성
    df = pd.read_parquet(f"../../data/view_{data}.parquet.gzip", columns=["month", "day", "hour", "category_code_0"])
    for category in big_category:
        category_df = df[df["category_code_0"] == category]
        temp_df = pd.concat([temp_df, category_df[["month", "day", "hour", "category_code_0"]].groupby(["month", "day","hour"]).count()], axis=1)
        temp_df.rename(columns={"category_code_0" : f"view_of_st_{category}"}, inplace = True)

    return temp_df

In [33]:
# 시간별 데이터 피클링
for data in data_list:
    df = convert_to_time_data(data)
    df.to_pickle(DATA_PATH + "groupby_time_" + data + ".pkl")
    del df

In [66]:
# 모든 시간별 데이터 하나로 합치고 저장
time_dataframe_10 = pd.read_pickle(DATA_PATH + "groupby_time_" + data_list[0] + ".pkl")
time_dataframe_11 = pd.read_pickle(DATA_PATH + "groupby_time_" + data_list[1] + ".pkl")
time_dataframe = pd.concat([time_dataframe_10, time_dataframe_11])
time_dataframe.to_pickle(DATA_PATH + "groupby_time_data.pkl")

# 시간별 데이터 전처리
df = pd.read_pickle(DATA_PATH + "groupby_time_data.pkl")
df = df.fillna(0)
df = df.reset_index()
df["time"] =  "2019" + "-"\
            + df["month"].apply(lambda x : "0" + str(x) if len(str(x)) == 1 else str(x)) + "-"\
            + df["day"].apply(lambda x : "0" + str(x) if len(str(x)) == 1 else str(x)) + "-"\
            + df["hour"].apply(lambda x : "0" + str(x) if len(str(x)) == 1 else str(x))
df["time"] = pd.to_datetime(df["time"])
df["week_day"] = df["time"].dt.weekday
columns = df.columns.to_list()
columns.insert(0, columns.pop())
columns.insert(0, columns.pop())
df = df[columns]         
df = df.sort_values(by = ["time"])
df = df.reset_index(drop=True)  
# 시간대 변수 두가지 생성(general, specific)
df["general_time_category"] = df["hour"].apply(to_hour_category, args=([6,12,18,24],))
df["specific_time_category"] = df["hour"].apply(to_hour_category, args=([4,12,19,22],))
columns = df.columns.to_list()
columns.insert(0, columns.pop())
columns.insert(0, columns.pop())
df = df[columns]
temp_dict = {1 : "morning", 2 : "afternoon", 3 : "evening", 0 : "bedtime"}
df["general_time_category"] = df["general_time_category"].apply(lambda x : temp_dict[x])
df["specific_time_category"] = df["specific_time_category"].apply(lambda x : temp_dict[x])
# 시간별 결제 인원의 비율    
df["no_of_view_to_purchase_user"] = df["no_of_purchase_user"] / df["no_of_total_user"]
# 시간별 장바구니에 담은 인원의 비율
df["no_of_view_to_cart_user"] = df["no_of_cart_user"] / df["no_of_total_user"]
# 시간별 장바구니에 담은 인원대비 결제하는 비중
df["no_of_cart_to_purchase_user"] = df["no_of_purchase_user"] / df["no_of_cart_user"]

# 방문이나 장바구니 담기는 있어도 단 한번의 결제 로그도 없는 날자가 존재한다. 해당 날자의 값은 이상치로 생각하고 제거한다.
# 제거되는 날 11월 15일
drop_index = df[(df["month"] == 11) & (df["day"] == 15)].index
df = df.drop(drop_index)
df = df.reset_index(drop=True)
# 12월의 데이터 삭제
drop_index = df[df["month"] == 12].index
df = df.drop(drop_index)
df = df.reset_index(drop=True)
df.to_pickle(DATA_PATH + "groupby_time_data.pkl")

In [7]:
# 시간별 데이터 최종완성본
df = pd.read_pickle(DATA_PATH + "groupby_time_data.pkl")
df

,general_time_category,specific_time_category,time,week_day,month,day,hour,event_type_view,event_type_purchase,event_type_cart,...,view_of_st_construction,view_of_st_kids,view_of_st_accessories,view_of_st_sport,view_of_st_medicine,view_of_st_country_yard,view_of_st_stationery,no_of_view_to_purchase_user,no_of_view_to_cart_user,no_of_cart_to_purchase_user
0,bedtime,morning,2019-10-01 04:00:00,1,10,1,4,1070,10,3,...,17,4.0,1.0,1.0,0.0,0.0,0.0,0.023499,0.007833,3.000000
1,bedtime,morning,2019-10-01 05:00:00,1,10,1,5,121,0,0,...,4,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,NaN
2,morning,morning,2019-10-01 06:00:00,1,10,1,6,22326,316,244,...,389,254.0,165.0,116.0,8.0,11.0,22.0,0.049833,0.029007,1.717949
3,morning,morning,2019-10-01 07:00:00,1,10,1,7,47951,845,613,...,1088,608.0,301.0,193.0,16.0,18.0,6.0,0.065912,0.036142,1.823684
4,morning,morning,2019-10-01 08:00:00,1,10,1,8,53390,1021,879,...,1035,702.0,348.0,235.0,17.0,15.0,2.0,0.070477,0.043158,1.633010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,evening,evening,2019-11-30 19:00:00,5,11,30,19,109774,1418,4396,...,1683,1526.0,634.0,434.0,67.0,52.0,26.0,0.052042,0.113105,0.460123
1432,evening,evening,2019-11-30 20:00:00,5,11,30,20,112070,1342,4220,...,1941,1530.0,701.0,408.0,64.0,4.0,13.0,0.051490,0.114714,0.448852
1433,evening,evening,2019-11-30 21:00:00,5,11,30,21,107025,1271,3793,...,1509,1582.0,859.0,495.0,38.0,37.0,25.0,0.051953,0.111449,0.466165
1434,evening,bedtime,2019-11-30 22:00:00,5,11,30,22,86724,939,2998,...,1381,1373.0,659.0,350.0,12.0,23.0,29.0,0.049957,0.109219,0.457402
